In [1]:
import time
import sys
from pynq import Overlay, MMIO, allocate
from pynq import MMIO
import numpy as np
ol = Overlay("pulseGen.bit")

class PulseGen():
    def __init__(self,GPIO_ADDRESS,GPIO_RANGE,DMA_ADDRESS,DMA_RANGE):
        self.gpio = MMIO(GPIO_ADDRESS, GPIO_RANGE)
        self.dma = MMIO(DMA_ADDRESS, DMA_RANGE)
        self.gpioDataOut=0
        
    def setResetn(self,val):
        mask=0b11111111111111111111111110
        self.gpioDataOut=(self.gpioDataOut & mask) | (val << 0)
        self.gpio.write(0, self.gpioDataOut)
        return
    def setTrig(self,val):
        mask=0b11111111111111111111111101
        self.gpioDataOut=(self.gpioDataOut & mask) | (val << 1)
        self.gpio.write(0, self.gpioDataOut)
        return
    def setPulseWidth(self,val):
        mask=0b00000000000000000000000011
        self.gpioDataOut=(self.gpioDataOut & mask) | (val << 2)
        self.gpio.write(0, self.gpioDataOut)
        return
    def getState(self):
        return self.gpio.read(0x0008) & 0b111
    
    def getStreamDownCounter(self):
        return (self.gpio.read(0x0008) >> 3)
        
    def dmaMM2SIsIdle(self):
        isIdle=bool(self.dma.read(0x4) & (1<<1))
        return isIdle
    
    def dmaMM2SConfig(self,bufferAddress):
        self.dma.write(0x18,bufferAddress)
        return
    
    def dmaMM2SRun(self,bufferBytesLen):
        self.dma.write(0x28,bufferBytesLen)
        return
    
    def dmaMM2SReset(self):
        self.dma.write(0x0,4)
        return
    
    def dmaMM2SHalt(self):
        self.dma.write(0x0,0)
        return 
    
    def dmaMM2SStart(self):
        self.dma.write(0x0,0x0001)

In [2]:
####################################################
# Parameters
####################################################
# Physical addresses
GPIO_ADDRESS=0x41200000
GPIO_RANGE=0x10000
DMA_ADDRESS=0x40400000
DMA_RANGE=0x10000

#Pulse parameters
PULSE_WIDTH=5 # In units of 10ns

#Pulse lists
counterList=[1,11,12,20,31,32,33] # Sarting point of each pulse. In units of 10ns. 
timestampList=[0,0,0,0,0,0,0]     # 8 bit timestamp of each pulse
maskList=[1,2,1,7,4,2,1]          # Port mask of each pulse


####################################################
# Instantiations and memory allocations
####################################################
pulseGen=PulseGen(GPIO_ADDRESS,GPIO_RANGE,DMA_ADDRESS,DMA_RANGE)      
bufferLen=len(counterList)
bufferType=np.uint64

buffer = allocate(bufferLen, dtype=bufferType)
bufferAddress=buffer.physical_address
bufferBytesLen=buffer.nbytes

####################################################
# Initial config
####################################################
pulseGen.setPulseWidth(PULSE_WIDTH)
pulseGen.dmaMM2SHalt()
pulseGen.dmaMM2SReset()
pulseGen.dmaMM2SConfig(bufferAddress)
pulseGen.dmaMM2SStart()

####################################################
# Load data into buffer
####################################################
for i in range(bufferLen):
    buffer[i]= (maskList[i] << 32) | (timestampList[i] << 24)  + counterList[i] 

####################################################
#  Run pulse generator
####################################################
pulseGen.setResetn(1)                        #Enable pulse generator core
pulseGen.dmaMM2SRun(bufferBytesLen)          #Run DMA engine
pulseGen.setTrig(1)                          #Generate trigger HIGH (starts pulse generator core)
pulseGen.setTrig(0)                          #Generate trigger HIGH
while(pulseGen.getState==(1<<1)): pass       #Wait for pulse generator to finish
print("Execution of pulse generator finished.\nStream down counter: {}\nCurrent state: {} (0->off 1->idle 2->run 4->error)".format(pulseGen.getStreamDownCounter(),pulseGen.getState()))


Execution of pulse generator finished.
Stream down counter: 7
Current state: 1 (0->off 1->idle 2->run 4->error)
